In [ ]:
from DeepFMKit.core import DeepFitFramework

In [ ]:
dff = DeepFitFramework()

label = "dynamic"
dff.new_sim(label)
dff.sims[label].m = 6.0
dff.sims[label].f_mod = 1000
dff.sims[label].f_samp = int(200e3)
dff.sims[label].f_n = 1e6
dff.sims[label].arml_mod_f = 1.0
dff.sims[label].arml_mod_amp = 1e-9
dff.sims[label].arml_mod_n = 1e-12
dff.sims[label].fit_n = 10

dff.simulate(label, n_seconds=10, simulate="dynamic", ref_channel=True)

dff.sims[label].info()

In [ ]:
for i, key in enumerate(dff.sims):
    dff.fit(label=key, fit_label=f'ch{i}', use_c_core=False)

In [ ]:
dff.plot()